### Importing require libraries

In [30]:
# Import necessary libraries
import torch
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [31]:
# Folder containing Excel files
folder_path = r'data\labeldata.parquet'

data = pd.read_parquet(folder_path)
data.head()

,id,subject,emailtext,label
0,738a2c78-e9b7-4886-8a1b-7af4c08f3906,RE: Yates Residence - Mechanical Contractor Me...,"Zac Stevenson, PEPRINCIPAL | BUILDING MEPC . ....",0
1,23e4febf-364f-464a-9a3a-80f9d89d0e6b,Re: 6771 - VeLa - Level 4 Slab Penetration Plans,You tooRespectfullyMichael BentleyGeneral Fore...,0
2,f0789ea4-0a5f-46c3-8adb-083766b63740,RE: FW: 13027A - 20240603 - Prl - Columbia + A...,You can update the two 12 ducts and adjust on ...,0
3,09474b0b-bc7f-4a1f-ab95-0ed2c497468a,RE: West Zephyrhills Elementary - Pasco County...,"You can start with Area A. Proceeded by B,C,D ...",0
4,b7438359-ce6c-42f7-bcfd-5690e875dcda,RE: MLW_0001_CRS_24410_ Connacht Stadium - Exi...,You can hold off on doing this for now.,0


In [32]:
df = data[['emailtext', 'label']].copy()
df = df.rename({'emailtext': 'text'}, axis=1)
df.head()

,text,label
0,"Zac Stevenson, PEPRINCIPAL | BUILDING MEPC . ....",0
1,You tooRespectfullyMichael BentleyGeneral Fore...,0
2,You can update the two 12 ducts and adjust on ...,0
3,"You can start with Area A. Proceeded by B,C,D ...",0
4,You can hold off on doing this for now.,0


In [33]:
df.label.value_counts()

label
0    5798
1      55
Name: count, dtype: int64

In [34]:
# 2. Split data into training and validation sets
train_df, val_df = train_test_split(
    df,
    test_size=0.1,
    stratify=df['label'],  # Ensures the split is stratified
    random_state=42
)

In [35]:
# Reset indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [36]:
# 3. Calculate class weights
# Assign class weights as variables in the code
labels = train_df['label'].values  # Assuming labels are 0 and 1

In [37]:
# Compute class weights using sklearn's compute_class_weight function
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

# # Convert class weights to a tensor
# class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights.tolist()

# Print class weights for verification
print(f"Class Weights: {class_weights}")

Class Weights: [0.5046952855500192, 53.744897959183675]


In [38]:
# 4. Define the custom dataset
class EmailDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),  # Flatten to remove extra dimension
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [39]:
# 5. Initialize tokenizer and model configuration
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128
num_labels = len(np.unique(labels))  # Should be 2 for labels 0 and 1

In [40]:
# Load the configuration and set class weights as a variable in the config
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=num_labels)
config.class_weights = class_weights  # Assign class_weights to the config

In [41]:
from transformers.modeling_outputs import SequenceClassifierOutput

class WeightedBertForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.class_weights = torch.tensor(config.class_weights, dtype=torch.float)

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        # Get the outputs from the base model without computing the default loss
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=None,  # Avoid default loss computation
            **kwargs
        )
        logits = outputs.logits

        # Compute custom loss with class weights
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Return outputs as a SequenceClassifierOutput
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [42]:
# 7. Instantiate the model using the custom class and config
model = WeightedBertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    config=config
)

Some weights of WeightedBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
# 8. Create training and validation datasets
train_dataset = EmailDataset(
    texts=train_df['text'],
    labels=train_df['label'],
    tokenizer=tokenizer,
    max_length=max_length
)

val_dataset = EmailDataset(
    texts=val_df['text'],
    labels=val_df['label'],
    tokenizer=tokenizer,
    max_length=max_length
)

In [44]:

# 9. Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',           # Output directory
    num_train_epochs=2,               # Adjust number of epochs as needed
    per_device_train_batch_size=5,    # Batch size for training
    per_device_eval_batch_size=5,     # Batch size for evaluation
    evaluation_strategy='epoch',      # Evaluate every epoch
    save_strategy='epoch',            # Save model every epoch
    logging_dir='./logs',             # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,      # Load the best model at the end of training
    metric_for_best_model='f1',       # Use F1 score to select best model
    greater_is_better=True,           # Higher F1 score is better
)

c:\Suraj\projects\emailsentiment\venv\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [45]:
# 10. Define the compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=1
    )
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [46]:

# 11. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [47]:
# 12. Train the model
trainer.train()

 20%|██        | 659/3295 [2:45:23<11:01:35, 15.06s/it]
                                                 
  0%|          | 10/2108 [00:14<50:04,  1.43s/it]  

{'loss': 0.2604, 'grad_norm': 0.9774286150932312, 'learning_rate': 4.976280834914611e-05, 'epoch': 0.01}


                                                   
  1%|          | 20/2108 [00:32<1:06:17,  1.91s/it]

{'loss': 0.0123, 'grad_norm': 0.09739227592945099, 'learning_rate': 4.952561669829222e-05, 'epoch': 0.02}


                                                   
  1%|▏         | 30/2108 [00:52<1:09:43,  2.01s/it]

...

100%|██████████| 2108/2108 [1:03:22<00:00,  1.74s/it]





















































































































                                                     

                                           
100%|██████████| 2108/2108 [1:04:20<00:00,  1.74s/it]



{'eval_loss': 0.3370947241783142, 'eval_accuracy': 0.9897610921501706, 'eval_f1': 0.0, 'eval_precision': 1.0, 'eval_recall': 0.0, 'eval_runtime': 57.0358, 'eval_samples_per_second': 10.274, 'eval_steps_per_second': 2.069, 'epoch': 2.0}


                                                     
100%|██████████| 2108/2108 [1:04:22<00:00,  1.83s/it]

{'train_runtime': 3862.1895, 'train_samples_per_second': 2.727, 'train_steps_per_second': 0.546, 'train_loss': 0.2868997352552314, 'epoch': 2.0}


TrainOutput(global_step=2108, training_loss=0.2868997352552314, metrics={'train_runtime': 3862.1895, 'train_samples_per_second': 2.727, 'train_steps_per_second': 0.546, 'total_flos': 692902964290560.0, 'train_loss': 0.2868997352552314, 'epoch': 2.0})

In [48]:
# 13. Evaluate the model
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

100%|██████████| 118/118 [00:43<00:00,  2.70it/s]

Evaluation Results: {'eval_loss': 0.37636786699295044, 'eval_accuracy': 0.9897610921501706, 'eval_f1': 0.0, 'eval_precision': 1.0, 'eval_recall': 0.0, 'eval_runtime': 44.1267, 'eval_samples_per_second': 13.28, 'eval_steps_per_second': 2.674, 'epoch': 2.0}


In [ ]:
def predict_email(email_text, threshold=0.3):
    inputs = tokenizer(
        email_text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_prob = probabilities[0][1].item()
    return 1 if predicted_prob >= threshold else 0

In [ ]:
# # 14. Save the trained model and tokenizer
# model.save_pretrained('your-trained-model')
# tokenizer.save_pretrained('your-trained-model')